In [1]:
!nvidia-smi

Thu Mar 21 10:16:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.26                 Driver Version: 546.26       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX450         WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P0              N/A / ERR! |      0MiB /  2048MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

You should consider upgrading via the 'D:\VisualStudioPython\Text Summarizer End to End Project\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

You should consider upgrading via the 'D:\VisualStudioPython\Text Summarizer End to End Project\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0
  Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


You should consider upgrading via the 'D:\VisualStudioPython\Text Summarizer End to End Project\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM , AutoTokenizer, AutoConfig
import nltk
from tqdm import tqdm
import torch
nltk.download('punkt')

d:\VisualStudioPython\Text Summarizer End to End Project\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
import os
os.environ['HF_HOME'] = 'D:/VisualStudioPython/Text Summarizer End to End Project/Text-Summarizer/hugging_face_model_downloaded'

In [ ]:
# Set the new cache directory
os.environ["TRANSFORMERS_CACHE"] = "/path/to/my/cache/directory"

In [4]:
model_ckpt= 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt,cache_dir="D:/VisualStudioPython/Text Summarizer End to End Project/Text-Summarizer/hugging_face_model_downloaded")
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt,cache_dir="D:/VisualStudioPython/Text Summarizer End to End Project/Text-Summarizer/hugging_face_model_downloaded")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import zipfile

zip_file_path = "D:/VisualStudioPython/Text Summarizer End to End Project/dataset/samsumdata.zip"
extract_to_directory = "D:/VisualStudioPython/Text Summarizer End to End Project/dataset/samsumdata"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents to the specified directory
    zip_ref.extractall(extract_to_directory)

In [6]:
dataset_samsum = load_from_disk('D:/VisualStudioPython/Text Summarizer End to End Project/dataset/samsumdata/samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [7]:
dataset_samsum['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [8]:
for i in dataset_samsum:
    print(i)
    print(len(dataset_samsum[i]))

train
14732
test
819
validation
818


In [9]:
split_lengths= [len(dataset_samsum[splits])for splits in dataset_samsum]

print(f"Split length = {split_lengths}")
print(f"Features : {dataset_samsum['train']}")

Split length = [14732, 819, 818]
Features : Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})


In [10]:
type(dataset_samsum['train'])

datasets.arrow_dataset.Dataset

In [11]:
len(dataset_samsum['train'])

14732

In [12]:
type(dataset_samsum['train'][1])

dict

In [13]:
dataset_samsum['train'][1]['dialogue']

'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great'

In [14]:
print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [15]:
config = AutoConfig.from_pretrained(pretrained_model_name_or_path="google/pegasus-cnn_dailymail")
config

PegasusConfig {
  "_name_or_path": "google/pegasus-cnn_dailymail",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "forced_eos_token_id": 1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "length_penalty": 0.8,
  "max_length": 128,
  "max_position_embeddings": 1024,
  "min_length": 3

## Working of tokenizer :

for more details read here , [text](https://huggingface.co/learn/nlp-course/chapter2/4?fw=pt)

In [16]:
tokens=tokenizer.tokenize("Using a Transformer network is simple")
tokens

['▁Using', '▁a', '▁Transformer', '▁network', '▁is', '▁simple']

In [17]:
ids=tokenizer.convert_tokens_to_ids(tokens)
ids

[3270, 114, 51979, 952, 117, 586]

In [18]:
decode=tokenizer.decode(ids)
decode

'Using a Transformer network is simple'

# Example of Tokenizer

In [19]:
# For Input
id =tokenizer("Using a Transformer network is simple" , max_length = 1024, truncation = True )

In [20]:
id['input_ids'], id['attention_mask']

([3270, 114, 51979, 952, 117, 586, 1], [1, 1, 1, 1, 1, 1, 1])

In [21]:
sequence = "Using a Transformer network is simple"

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[3270, 114, 51979, 952, 117, 586, 1]
[3270, 114, 51979, 952, 117, 586]


In [22]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

Using a Transformer network is simple</s>
Using a Transformer network is simple


In [23]:
# For target 
with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer("Using a Transformer network is simple", max_length = 128, truncation = True )
        print(target_encodings)

{'input_ids': [3270, 114, 51979, 952, 117, 586, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


d:\VisualStudioPython\Text Summarizer End to End Project\myenv\lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Actual Code

In [24]:
def convert_examples_to_features(example_batch):

    input_encodings = tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)
    

    return {
    "input_ids": input_encodings['input_ids'],
    "attention_mask": input_encodings['attention_mask'],
    "labels": target_encodings['input_ids']

}

In [25]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

In [26]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [27]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [28]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [29]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])

d:\VisualStudioPython\Text Summarizer End to End Project\myenv\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
trainer.train()

  1%|          | 10/920 [12:45<16:20:49, 64.67s/it]

{'loss': 3.1995, 'grad_norm': 20.161909103393555, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/920 [24:42<18:54:13, 75.62s/it]

{'loss': 3.2262, 'grad_norm': 9.775856971740723, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


  3%|▎         | 30/920 [35:06<15:23:37, 62.27s/it]

...

{'loss': 1.7858, 'grad_norm': 3.896730422973633, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.28}


 29%|██▉       | 270/920 [4:09:47<8:55:49, 49.46s/it] 

{'loss': 1.7234, 'grad_norm': 4.130237102508545, 'learning_rate': 2.7000000000000002e-05, 'epoch': 0.29}


 30%|██▉       | 274/920 [4:13:00<8:39:54, 48.29s/it]

KeyboardInterrupt: 